# Import MODFLOW 6 models from Geomodelr

Welcome to Geomodelr webinar. Here we are going to explain how to load and run our models created and exported from Geomodelr. Furthemore, we will show how to get the results and save them in the _vtk_ file.

For this goal, we need the last version of flopy (3.2.12). Furthemore, we are using the vtk library, version 8.2.0 and numpy, version 1.16.04

In [1]:
import numpy as np
import flopy
import vtk
import json
from vtk.util import numpy_support

print('numpy version {}'.format(np.__version__))
print('flopy version {}'.format(flopy.__version__))
print('json version {}'.format(json.__version__))
print(vtk.vtkVersion.GetVTKSourceVersion())

flopy is installed in D:\Programas_Instalados\Anaconda3\envs\advanced-math\lib\site-packages\flopy
numpy version 1.16.4
flopy version 3.2.12
json version 2.0.9
vtk version 8.2.0


## Load simulation

We define the name of the simulation and its location. On the other hand, we must to set the location of mf6.exe (including the executable)

In [2]:
file_name = 'mf6_webinar' # Name of the study

sim_location = 'D:/CompartidaVB/Modflow/Webinar_2019/modflow6/{}/'.format(file_name)
exe_location = 'C:/WRDAPP/mf6.0.4/mf6.0.4/bin/mf6.exe' # mf6 exucatable location

After that, we load the simulation

In [3]:
sim = flopy.mf6.MFSimulation.load(sim_ws=sim_location , exe_name=exe_location)
model = sim.get_model(file_name)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disu...
    loading package riv...
    loading package wel...
    loading package npf...
    loading package ic...
    loading package oc...
  loading ims package mf6_webinar...


In [4]:
model_packages = model.package_names #All packages of the model
print('Model packges: {}\n'.format(model_packages))

sim_packages = sim.package_key_dict.keys()
print('Simulation packges: {}\n'.format(sim_packages))


Model packges: ['disu', 'riv_0', 'wel', 'npf', 'ic', 'oc']

Simulation packges: dict_keys(['nam', 'tdis', 'ims'])



### DISU Package
It is the package that contains all the data about mesh.


In [5]:
disu = model.get_package('disu')

print('Vertices info:\n\n{}'.format(disu.vertices.array)) # vertices array

Vertices info:

[(    0, 1067350.63, 1073405.91) (    1, 1067333.06, 1073406.41)
 (    2, 1067317.78, 1073393.55) ... (52117, 1066736.15, 1063707.81)
 (52118, 1066748.39, 1063707.81) (52119, 1066760.63, 1063707.81)]


In [6]:
print('Cells info:\n\n{}'.format(disu.cell2d.array)) # vertices array

Cells info:

[(     0, 1067330.93, 1073391.66, 6,     6,     5,     4,     3, 2, 1, None, None, None, None)
 (     1, 1067330.93, 1073391.66, 6,     6,     5,     4,     3, 2, 1, None, None, None, None)
 (     2, 1067330.93, 1073391.66, 6,     6,     5,     4,     3, 2, 1, None, None, None, None)
 ...
 (339120, 1066766.75, 1063713.32, 4, 49512, 52120, 47448, 47449, None, None, None, None, None, None)
 (339121, 1066766.75, 1063713.32, 4, 49512, 52120, 47448, 47449, None, None, None, None, None, None)
 (339122, 1066766.75, 1063713.32, 4, 49512, 52120, 47448, 47449, None, None, None, None, None, None)]


In [7]:
print('Conection info:\t{}'.format(disu.ja.array)) # vertices array

Conection info:	[    -1      2     11 ... 339109 339110 339122]


## Riv package
This package contains all data about rivers: river nodes, conductance, stage and bottom of the rivers.

In [8]:
riv = model.get_package('riv')
print(riv.stress_period_data)

{internal}
([((1080,), 2615.56543162, 30., 2613.56543162, 'cd_las_lara')
 ((1091,), 2603.10831491, 30., 2601.10831491, 'cd_las_lara')
 ((1103,), 2598.00491308, 30., 2596.00491308, 'cd_las_lara') ...
 ((9850,), 1918.98375071, 30., 1916.98375071, 'q._los_confines')
 ((330608,), 1916.61170247, 30., 1914.61170247, 'q._los_confines')
 ((330622,), 1920.30177343, 30., 1918.30177343, 'q._los_confines')])



## Wel package
This package contains all data about wells: wells nodes, volumetric well rate and ther names

In [9]:
wel = model.get_package('wel')
print(wel.stress_period_data)

{internal}
([((21427,), -18., 'w7') ((21428,), -18., 'w7') ((21429,), -18., 'w7')
 ((21430,), -18., 'w7') ((21431,), -18., 'w7') ((21432,), -18., 'w7')
 ((21433,), -18., 'w7') ((21434,), -18., 'w7') ((21435,), -18., 'w7')
 ((21436,), -18., 'w7') ((21437,), -18., 'w7') ((21438,), -18., 'w7')
 ((21439,), -18., 'w7') ((21440,), -18., 'w7') ((21441,), -18., 'w7')
 ((21442,), -18., 'w7') ((21443,), -18., 'w7') ((21444,), -18., 'w7')
 ((21445,), -18., 'w7') ((21446,), -18., 'w7') ((21447,), -18., 'w7')
 ((21470,), -17., 'w6') ((21471,), -17., 'w6') ((21472,), -17., 'w6')
 ((21473,), -17., 'w6') ((21474,), -17., 'w6') ((21475,), -17., 'w6')
 ((21476,), -17., 'w6') ((21477,), -17., 'w6') ((21478,), -17., 'w6')
 ((21479,), -17., 'w6') ((21480,), -17., 'w6') ((21481,), -17., 'w6')
 ((21482,), -17., 'w6') ((21483,), -17., 'w6') ((21484,), -17., 'w6')
 ((21520,), -16., 'w5') ((21521,), -16., 'w5') ((21522,), -16., 'w5')
 ((21523,), -16., 'w5') ((21524,), -16., 'w5') ((21525,), -16., 'w5')
 ((21526

# IMS package (Iterative Model Solution)
This package contains the information related about numerical solver. This package is registers to simulation and it is assigned to each model.

In [10]:
ims = sim.get_package('ims')
print(ims)

package_name = mf6_webinar
filename = mf6_webinar.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = modflowsim

Block options
--------------------
print_option
{internal}
(all)

complexity
{internal}
(moderate)


Block nonlinear
--------------------
outer_hclose
{internal}
(0.01)

outer_maximum
{internal}
(50)


Block linear
--------------------
inner_maximum
{internal}
(30)

inner_hclose
{internal}
(0.01)

linear_acceleration
{internal}
(cg)

relaxation_factor
{internal}
(0.97)

preconditioner_levels
{internal}
(8)

preconditioner_drop_tolerance
{internal}
(0.0001)





In [11]:
# Maximum number of outer (nonlinear) iterations.
print('Maximum number of nonlinear interations: {}'.format(ims.outer_maximum.get_data()))
ims.outer_maximum.set_data(500)

# Maximum number of inner (linear) iterations.
print('Maximum number of linear interations: {}'.format(ims.inner_maximum.get_data()))
ims.inner_maximum.set_data(130)

Maximum number of nonlinear interations: 50
Maximum number of linear interations: 30


In [12]:
# Maximum number of outer (nonlinear) iterations.
print('Maximum number of nonlinear interations: {}'.format(ims.outer_maximum.get_data()))

# Maximum number of inner (linear) iterations.
print('Maximum number of linear interations: {}'.format(ims.inner_maximum.get_data()))


Maximum number of nonlinear interations: 500
Maximum number of linear interations: 130


In [13]:
# Complexity: Defines if the model can trates as linear or nonlinear problem.
print('Complexity: {}'.format(ims.complexity.get_data()))
# simple: Model can be trated as linear problem (confined units, linear stress packages, etc.)

Complexity: moderate


In [14]:
ims.complexity.set_data('moderate')
print('Complexity: {}'.format(ims.complexity.get_data()))
# moderate (several unconfined units, nonlinear stress packages, etc.)

Complexity: moderate


# Newton-Raphson solver
This solver is recommended where traditional wet/drying numerical schemes does not get an acceptable solution due to convergence problems.

In [15]:
# Newton-Rhapson solver using under relaxation 
model.name_file.newtonoptions = ('UNDER_RELAXATION')
# This option allows to under-relax the head where water level falls below bottom of cell.

# Run the simulation

In [16]:
# write simulation
sim.write_simulation()

#run simulation
sim.run_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package mf6_webinar...
  writing model mf6_webinar...
    writing model name file...
    writing package disu...
    writing package riv_0...
    writing package wel...
    writing package npf...
    writing package ic...
    writing package oc...
FloPy is using the following  executable to run the model: C:/WRDAPP/mf6.0.4/mf6.0.4/bin/mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.0.4 03/13/2019

   MODFLOW 6 compiled Mar 13 2019 12:37:09 with IFORT compiler (ver. 19.0.0)

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or 

(False, [])

In [17]:
# Change linear acceleration method 
ims.linear_acceleration.set_data('BICGSTAB')
# write simulation
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package mf6_webinar...
  writing model mf6_webinar...
    writing model name file...
    writing package disu...
    writing package riv_0...
    writing package wel...
    writing package npf...
    writing package ic...
    writing package oc...


In [18]:
#run simulation
sim.run_simulation()

FloPy is using the following  executable to run the model: C:/WRDAPP/mf6.0.4/mf6.0.4/bin/mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.0.4 03/13/2019

   MODFLOW 6 compiled Mar 13 2019 12:37:09 with IFORT compiler (ver. 19.0.0)

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Government shall be held liable for any damages resulting from its 
authorized or unauthorized use. Also refer to the USGS W

(True, [])

# Open results and save it on _vtk_ file
The hydraulic heads data are saved on the _.hds_ file. We can get this data using the output_keys command.

In [19]:
keys = sim.simulation_data.mfdata.output_keys()

('mf6_webinar', 'CBC', 'FLOW-JA-FACE')
('mf6_webinar', 'CBC', 'WEL')
('mf6_webinar', 'CBC', 'RIV')
('mf6_webinar', 'HDS', 'HEAD')


In [20]:
# get all head data
head = sim.simulation_data.mfdata[file_name, 'HDS', 'HEAD']
head = head[0]
print(head)

[2569.77268033 2565.46390276 2560.90141369 ... 2294.80674201 2291.68578815
 2288.08941353]


It should be noted that when Newton-Raphson method is used no cells will dry. Consequently, we have to mark all cell where head is below its bottom. For this, we create a numpy array where val=1 if head<bottom (dry cell) and val=0 if not.

In [21]:
disu = model.get_package('disu') #First, we get DISU package from the model.
bot = disu.bot.array # Second, we get cells bottom from DISU package.
#mask = head < bot # Mask where val=True if head<bottom and val=False if not.

dry_cells = np.array( head - bot, dtype='float32')

_dry-cells_ is the numpy array where val=1 if head<bottom (dry cell) and val=0 if not. Now, we are ready to load the _vtk_ file and add the head and dry data to visalizate in Paraview.

In [22]:
# Read the vtk source file.
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName(sim_location + file_name + '.vtk')
reader.Update() # Needed because of GetScalarRange
ugrid = reader.GetOutput()
reader.CloseVTKFile()

# Convert head array to vtk_array and save it on untructured grid (ugrid)
heads = numpy_support.numpy_to_vtk(head.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
heads.SetName('Heads')
ugrid.GetCellData().AddArray(heads)

# Convert dry_cells array to vtk_array and save it on untructured grid (ugrid)
dry_cells = numpy_support.numpy_to_vtk(dry_cells.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
dry_cells.SetName('Dry')
ugrid.GetCellData().AddArray(dry_cells)

8

Finally, we update the _vtk_ file.

In [23]:
writerVTK = vtk.vtkUnstructuredGridWriter()
writerVTK.SetInputData(ugrid)
writerVTK.SetFileName(sim_location + file_name + '.vtk')
writerVTK.Update()

# Reading units nodes
Geomodelr exports a _.json_ file that containts a dictionary where the keys are the geological units names and their values correponds to id nodes of the mesh.

In [24]:
json_file = sim_location + file_name + '.json'
print('JSON file name:\n{}\n'.format(json_file))

with open(json_file) as json_file:  
    unit_nodes = json.load(json_file)

print('Geological units: {}'.format(unit_nodes.keys()))
print('Number of nodes of E2p: {}'.format(len(unit_nodes[u'K2p'])))
print('Number of nodes of E1ss: {}'.format(len(unit_nodes[u'E1ss'])))
print('Number of nodes of E2E3co: {}'.format(len(unit_nodes[u'E2E3co'])))
print('\nFirst 20 nodes of E2E3co: {}'.format(unit_nodes[u'E2E3co'][:10]))

JSON file name:
D:/CompartidaVB/Modflow/Webinar_2019/modflow6/mf6_webinar/mf6_webinar.json

Geological units: dict_keys(['E2p', 'K2p', 'E1ss', 'K2E1g', 'K2lt', 'E1si', 'K2cp', 'E2E3co', 'E1c', 'K2d'])
Number of nodes of E2p: 44562
Number of nodes of E1ss: 18317
Number of nodes of E2E3co: 2866

First 20 nodes of E2E3co: [2649, 2650, 2671, 3475, 3494, 3513, 3532, 3533, 3552, 3553]
